In [1]:
import sys
import os
import warnings
import re 
import argparse
import subprocess
import pandas as pd
from pandas import isna 
import numpy as np

from tempfile import NamedTemporaryFile

from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
import concurrent.futures
from scipy.cluster.hierarchy import fclusterdata
from typing import Tuple
from functools import partial
import csv 

from typing import Dict

### local imports 
sys.path.append("/home/150/as7425/R1/read_classifier")
from bam_to_bed import bam_to_bed
from process_genome import generate_genome_file, read_chromosomes_from_genome_file, filter_bed_by_chromosome_inplace
from gtf_to_bed import gtf_to_bed, extend_gene_bed 
from parallel_bed_operations import split_bed_file, write_sorted_chunk
from intersect import run_bedtools
from parse_intersect import parse_output
from process_read_end_positions import calculate_distance_for_unique_positions, calculate_distance_to_annotated_ends
from process_gtf import parse_attributes, parse_gtf_line, get_gene_exon_table, get_biotypes 
from parse_classifications import parse_read_classification, summarize_error_cases, classify_read, print_classification_summary


# set logging level 
import logging 
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')



In [2]:
# main 
def main(bam_file, gtf_file, output_table):

    # fetch output directory 
    output_dir = os.path.dirname(output_table) 

    # generate a genome file for bedtools sort 
    genome_file = generate_genome_file(bam_file, output_dir) 

    # create a bed file of gene regions
    gene_bed_file = gtf_to_bed(gtf_file, "gene", output_dir) 

    # filter for relevant chromsoomes
    chromosomes = read_chromosomes_from_genome_file(genome_file)
    filter_bed_by_chromosome_inplace(gene_bed_file, chromosomes)

    # create a bed file of downstream of gene regions 
    dog_bed_file = extend_gene_bed(gene_bed_file, output_dir, genome_file) 

    # intersect reads against, exons, genes, and downstream of gene regions 
    intersect_files = run_bedtools(bam_file, gtf_file, dog_bed_file, genome_file, output_dir) 

    # parse the bedtools intersect files 
    df = parse_output(intersect_files[0], intersect_files[1], intersect_files[2], intersect_files[3], intersect_files[4])
    
    # Write df to a CSV file for testing
    df.to_csv(output_dir + "/test_df_end.csv", index=False)
    
    # fetch distances to nearest transcript end sites 
    end_position_df = calculate_distance_to_annotated_ends(df, gtf_file, output_dir)

    # save the output as CSV 
    end_position_df.to_csv(output_table, sep="\t", index=False)

    # fetch the gene biotypes 
    biotypes = get_biotypes(gtf_file)

    # classify each read 
    result_df = parse_read_classification(output_table, biotypes)

    # print some summary statistics 
    print_classification_summary(result_df)

    # Rename 'exon_count' to 'annotated_exon_count'
    result_df.rename(columns={'exon_count': 'annotated_exon_count'}, inplace=True)

    # Omit specific columns
    columns_to_omit = ['read_end_strand', 'strand_sign', 'chromosome', 'position', 'strand', 'biotypes_info']
    result_df.drop(columns=columns_to_omit, inplace=True)

    # Save the table to output
    result_df.to_csv(output_table, sep='\t', index=False)


In [3]:
# run on a subset of data 

# filtered read subsets from ASR012 
bam_file = "/g/data/lf10/as7425/2023_mrna-biogenesis-maps/analysis/2024-03-28_ASR012_HeLa-total-RNA004-rep1/ASR012_HeLa-total-RNA004/classify_reads_tests/filtered_reads.bam"

gtf_file = "/g/data/lf10/as7425/2023_mrna-biogenesis-maps/analysis/2024-03-28_ASR012_HeLa-total-RNA004-rep1/ASR012_HeLa-total-RNA004/classify_reads_tests/gene_annotation.gtf"

output_table = "/g/data/lf10/as7425/2023_mrna-biogenesis-maps/analysis/2024-03-28_ASR012_HeLa-total-RNA004-rep1/ASR012_HeLa-total-RNA004/classify_reads_tests/out/read_classification.txt"

main(bam_file, gtf_file, output_table)

2024-05-07 14:37:59 - INFO - Generating genome file
2024-05-07 14:37:59 - INFO - Converting gene GTF to BED
[E::idx_find_and_load] Could not retrieve index file for '/g/data/lf10/as7425/2023_mrna-biogenesis-maps/analysis/2024-03-28_ASR012_HeLa-total-RNA004-rep1/ASR012_HeLa-total-RNA004/classify_reads_tests/filtered_reads.bam'


   0         1         2                3   4  5
0  6  36605600  36606100  ENSG00000112081 NaN  +
{'1': 248956422, '10': 133797422, '11': 135086622, '12': 133275309, '13': 114364328, '14': 107043718, '15': 101991189, '16': 90338345, '17': 83257441, '18': 80373285, '19': 58617616, '2': 242193529, '20': 64444167, '21': 46709983, '22': 50818468, '3': 198295559, '4': 190214555, '5': 181538259, '6': 170805979, '7': 159345973, '8': 145138636, '9': 138394717, 'GL000008.2': 209709, 'GL000009.2': 201709, 'GL000194.1': 191469, 'GL000195.1': 182896, 'GL000205.2': 185591, 'GL000208.1': 92689, 'GL000213.1': 164239, 'GL000214.1': 137718, 'GL000216.2': 176608, 'GL000218.1': 161147, 'GL000219.1': 179198, 'GL000220.1': 161802, 'GL000221.1': 155397, 'GL000224.1': 179693, 'GL000225.1': 211173, 'GL000226.1': 15008, 'KI270302.1': 2274, 'KI270303.1': 1942, 'KI270304.1': 2165, 'KI270305.1': 1472, 'KI270310.1': 1201, 'KI270311.1': 12399, 'KI270312.1': 998, 'KI270315.1': 2276, 'KI270316.1': 1444, 'KI270317.1':

2024-05-07 14:38:01 - INFO - Converting BAM to BED: 1/104
2024-05-07 14:38:01 - INFO - Converting BAM to BED: 2/104
2024-05-07 14:38:01 - INFO - Converting BAM to BED: 3/104
2024-05-07 14:38:01 - INFO - Converting BAM to BED: 4/104
2024-05-07 14:38:01 - INFO - Converting BAM to BED: 5/104
2024-05-07 14:38:01 - INFO - Converting BAM to BED: 6/104
2024-05-07 14:38:01 - INFO - Converting BAM to BED: 7/104
2024-05-07 14:38:02 - INFO - Converting BAM to BED: 8/104
2024-05-07 14:38:02 - INFO - Converting BAM to BED: 9/104
2024-05-07 14:38:02 - INFO - Converting BAM to BED: 10/104
2024-05-07 14:38:02 - INFO - Converting BAM to BED: 11/104
2024-05-07 14:38:02 - INFO - Converting BAM to BED: 12/104
2024-05-07 14:38:02 - INFO - Converting BAM to BED: 13/104
2024-05-07 14:38:02 - INFO - Converting BAM to BED: 14/104
2024-05-07 14:38:02 - INFO - Converting BAM to BED: 15/104
2024-05-07 14:38:02 - INFO - Converting BAM to BED: 16/104
2024-05-07 14:38:02 - INFO - Converting BAM to BED: 17/104
2024-0

   0         1         2                3   4  5
0  6  36605600  36606100  ENSG00000112081 NaN  +
{'1': 248956422, '10': 133797422, '11': 135086622, '12': 133275309, '13': 114364328, '14': 107043718, '15': 101991189, '16': 90338345, '17': 83257441, '18': 80373285, '19': 58617616, '2': 242193529, '20': 64444167, '21': 46709983, '22': 50818468, '3': 198295559, '4': 190214555, '5': 181538259, '6': 170805979, '7': 159345973, '8': 145138636, '9': 138394717, 'GL000008.2': 209709, 'GL000009.2': 201709, 'GL000194.1': 191469, 'GL000195.1': 182896, 'GL000205.2': 185591, 'GL000208.1': 92689, 'GL000213.1': 164239, 'GL000214.1': 137718, 'GL000216.2': 176608, 'GL000218.1': 161147, 'GL000219.1': 179198, 'GL000220.1': 161802, 'GL000221.1': 155397, 'GL000224.1': 179693, 'GL000225.1': 211173, 'GL000226.1': 15008, 'KI270302.1': 2274, 'KI270303.1': 1942, 'KI270304.1': 2165, 'KI270305.1': 1472, 'KI270310.1': 1201, 'KI270311.1': 12399, 'KI270312.1': 998, 'KI270315.1': 2276, 'KI270316.1': 1444, 'KI270317.1':

/home/150/as7425/R1/read_classifier/parse_intersect.py:15: UserWarning: Warning: Essential file /g/data/lf10/as7425/2023_mrna-biogenesis-maps/analysis/2024-03-28_ASR012_HeLa-total-RNA004-rep1/ASR012_HeLa-total-RNA004/classify_reads_tests/out/tmpdmxekfxv_final.bed_dog_overlap_final.bed is empty. Some data may not be processed.
  warnings.warn(f"Warning: Essential file {file} is empty. Some data may not be processed.")


...

2024-05-07 14:38:09 - INFO - ran calculate_distance_for_unique_positions
2024-05-07 14:38:09 - INFO - Getting biotypes from GTF file
2024-05-07 14:38:09 - INFO - Starting to parse GTF file.
2024-05-07 14:38:09 - INFO - GTF file parsed. Processing data.
2024-05-07 14:38:09 - INFO - Data processed successfully. Returning final DataFrame.
2024-05-07 14:38:09 - ERROR - Exception in row read_id                             1cbd881b-4843-4427-a295-9e391a01195c
gene_id                                                  ENSG00000112081
read_length                                                         2049
alignment_length                                                    1950
splice_count                                                           5
gene_base_overlap                                                   1438
exon_base_overlap                                                   1438
intronic_alignment                                                     0
DOG_overlap                    

head of the gtf DataFrame:
    seqname          source feature     start       end score strand frame  \
0        6  ensembl_havana    gene  36594353  36605600     .      +     .   

                                           attribute  \
0  gene_id "ENSG00000112081"; gene_version "17"; ...   

                                          attributes          gene_id  \
0  {'gene_id': 'ENSG00000112081', 'gene_version':...  ENSG00000112081   

     gene_biotype  
0  protein_coding  

summary stats:
number of unique genes:  1
number of unique gene biotypes:  1


2024-05-07 14:38:09 - ERROR - Exception in row read_id                             abc0f89c-00e9-44ea-9334-b6dbf279b083
gene_id                                                  ENSG00000112081
read_length                                                         1579
alignment_length                                                    1486
splice_count                                                           5
gene_base_overlap                                                   1486
exon_base_overlap                                                   1486
intronic_alignment                                                     0
DOG_overlap                                                            0
unclassified_length                                                    0
unaligned_length                                                      93
read_end_chromosome                                                    6
read_end_position                                               36602929
read

Rows with exceptions or errors: 0

Read Classification Summary Stats:
unclassified    945
Name: classification, dtype: int64

Single Exon Summary Stats:
percentage of single exon reads: 0.0%
partially_transcribed: 0 (0.0%)
processed: 0 (0.0%)
readthrough: 0 (0.0%)
readthrough_cleaved: 0 (0.0%)
noncanonical_splicing_partially_transcribed: 0 (0.0%)
noncanonical_splicing_processed: 0 (0.0%)
noncanonical_splicing_readthrough: 0 (0.0%)
single-exon_unclassified: 0 (0.0%)

Multi Exon Summary Stats:
percentage of multi exon reads: 0.0%
readthrough_spliced: 0 (0.0%)
readthrough_partially_spliced: 0 (0.0%)
readthrough_ambiguous: 0 (0.0%)
readthrough_unspliced: 0 (0.0%)
readthrough_cleaved: 0 (0.0%)
cleaved_spliced: 0 (0.0%)
cleaved_partially_spliced: 0 (0.0%)
cleaved_ambiguous: 0 (0.0%)
cleaved_unspliced: 0 (0.0%)
partially_transcribed_spliced: 0 (0.0%)
partially_transcribed_partially_spliced: 0 (0.0%)
partially_transcribed_ambiguous: 0 (0.0%)
partially_transcribed_unspliced: 0 (0.0%)
multiexon_

KeyError: "['chromosome' 'position' 'strand'] not found in axis"

In [ ]:
if __name__ == "__main__":
    
    parser = argparse.ArgumentParser(description='Classify chromatin-bound direct RNA-Seq reads for splicing and transcriptional status')
    parser.add_argument('-b', '--bam_file', type=str, help='Path to the input bam file.')
    parser.add_argument('-g', '--gtf_file', type=str, help='Path to the GTF file.')
    parser.add_argument('-o', '--output_table', type=str, default=None, help='Optional path to the output file.')

    args = parser.parse_args()

    bam_file = args.bam_file
    gtf_file = args.gtf_file 
    output_table = args.output_table

    main(bam_file, gtf_file, output_table)


###### bam_file = "/g/data/lf10/as7425/2023_mrna-biogenesis-maps/analysis/2024-03-20_ASR011_HeLa-POINT-RNA004-rep1/ASR011_HeLa-POINT-RNA004-rep1_primary_genome_alignments_modCalled.bam"
gtf_file = "/g/data/lf10/as7425/genomes/human_genome/Homo_sapiens_transcriptsOnly_GRCh38.104.chr.gtf"
output_table = "/g/data/lf10/as7425/2023_mrna-biogenesis-maps/analysis/2024-03-20_ASR011_HeLa-POINT-RNA004-rep1/classify_reads/read_classification.txt"

main(bam_file, gtf_file, output_table)